<a href="https://colab.research.google.com/github/sahil3Vedi/Face-Mask-Classifier/blob/master/Face_Mask_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2,os
import numpy as np
from keras.utils import np_utils

data_path='drive/My Drive/Face Mask Detection'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))

data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
            resized=cv2.resize(gray,(100, 100))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:',e)
            

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],100, 100,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

DATA_LOC = 'drive/My Drive/Face Mask Detection/data.npy'
TARGET_LOC = 'drive/My Drive/Face Mask Detection/target.npy'

np.save(DATA_LOC,data)
np.save(TARGET_LOC,new_target)

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [4]:
model=Sequential()

model.add(Conv2D(100,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [5]:
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

checkpoint = ModelCheckpoint(
  'model-{epoch:03d}.model',
  monitor='val_loss',
  verbose=0,
  save_best_only=True,
  mode='auto')

history=model.fit(
  train_data,
  train_target,
  epochs=20,
  callbacks=[checkpoint],
  validation_split=0.2)

print(model.evaluate(test_data,test_target))

Train on 990 samples, validate on 248 samples
Epoch 1/20
990/990 [==============================] - 45s 45ms/step - loss: 0.7649 - accuracy: 0.5465 - val_loss: 0.6537 - val_accuracy: 0.5202
Epoch 2/20
990/990 [==============================] - 49s 49ms/step - loss: 0.6050 - accuracy: 0.6788 - val_loss: 0.5161 - val_accuracy: 0.7097
Epoch 3/20
990/990 [==============================] - 46s 47ms/step - loss: 0.4474 - accuracy: 0.8030 - val_loss: 0.3709 - val_accuracy: 0.8387
Epoch 4/20
990/990 [==============================] - 45s 46ms/step - loss: 0.3350 - accuracy: 0.8535 - val_loss: 0.2574 - val_accuracy: 0.9153
Epoch 5/20
990/990 [==============================] - 49s 49ms/step - loss: 0.2470 - accuracy: 0.9121 - val_loss: 0.2090 - val_accuracy: 0.9073
Epoch 6/20
990/990 [==============================] - 45s 46ms/step - loss: 0.1733 - accuracy: 0.9384 - val_loss: 0.1735 - val_accuracy: 0.9315
Epoch 7/20
990/990 [==============================] - 47s 47ms/step - loss: 0.1432 - accur

In [7]:
MODEL_LOC = 'drive/My Drive/Face Mask Detection/facemask_classifier.model'
model.save(MODEL_LOC)